In [1]:
from util.ImgMapper import *
import pandas as pd
from PIL import Image
from sklearn.decomposition import PCA
from pylab import *
from io import BytesIO
from scipy.misc import imsave, toimage
import base64

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days
Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


In [2]:
df = pd.read_csv("../features/BVLCref_layers/BVLCref_fc8.csv")
df.head()

,Unnamed: 0,fc8.0,fc8.1,fc8.2,fc8.3,fc8.4,fc8.5,fc8.6,fc8.7,fc8.8,...,fc8.990,fc8.991,fc8.992,fc8.993,fc8.994,fc8.995,fc8.996,fc8.997,fc8.998,fc8.999
0,../data/Rothko_images/AROTHKO_10310192985.jpg,-3.805288,-0.761124,-0.023702,-0.919209,-0.063642,0.544321,-0.955755,-3.986025,-3.965271,...,1.479064,-2.415221,-1.929235,-3.015062,-2.980834,-1.311198,-2.405202,-1.679754,-1.566092,2.735568
1,../data/Rothko_images/AROTHKO_10310800290.jpg,-2.085294,7.793644,-1.165780,-0.630009,-0.742318,-0.178394,-0.950502,-2.685611,-0.635306,...,-2.488261,1.567201,1.292316,-1.824902,-0.248830,-1.195391,2.722381,-2.064558,2.524389,6.639421
2,../data/Rothko_images/AMICO_DALLAS_103842196.jpg,-2.220360,2.204751,-3.474112,-2.867231,-3.790855,0.475245,-2.226392,-2.907528,-2.669872,...,3.294321,-0.247078,-1.086575,-0.003492,-0.230768,0.889488,0.719667,0.070013,1.787303,1.865200
3,../data/Rothko_images/AROTHKO_10313269612.jpg,-2.311052,0.698374,-1.019830,-2.154740,0.134849,-2.556587,-1.707413,-2.584717,-5.733024,...,-4.199751,-0.114115,-0.976569,-3.925595,-1.763975,-1.362928,0.726215,2.488721,5.835283,2.009402
4,../data/Rothko_images/AROTHKO_10312561458.jpg,-2.181365,1.577515,-2.397377,-1.784317,-3.032737,0.014922,-1.632777,-2.584925,-2.180451,...,-0.008083,-0.375961,0.770526,-2.487267,-0.155179,-1.006359,-0.793978,-1.459443,1.621819,4.188722


In [3]:
image_paths = list(df.iloc[:,0])
print len(image_paths)

415


In [4]:
tooltip_s = []

for image_path in image_paths:
    output = BytesIO()
    #img = toimage(image_data.reshape((28,28))) # Data was a flat row of 64 "pixels".
    img = Image.open(image_path).resize((100,100))
    img.save(output, format="PNG")
    contents = "<img src='data:image/png;base64,"+base64.encodestring(output.getvalue()).decode()+"'>"
    tooltip_s.append( contents )
    output.close()

tooltip_s = np.array(tooltip_s) # need to make sure to feed it as a NumPy array, not a list

In [6]:
image_features = array(df.iloc[:,1:])
image_features.shape

(415, 1000)

In [19]:
### Reduce dimensions and apply clustering

pca = PCA(n_components=50)
data = pca.fit_transform(array(image_features))

# Initialize to use t-SNE with 2 components (reduces data to 2 dimensions). Also note high overlap_percentage.
#mapper = KeplerMapper(clusterer=cluster.DBSCAN(eps=0.3, min_samples=15), nr_cubes=35, overlap_perc=0.9)
mapper = KeplerMapper()

In [20]:
# Fit and transform data
projected_data = mapper.fit_transform(data, projection=[0,1]) # X-Y axis


..Projecting data using: [0, 1]

..Scaling with: MinMaxScaler(copy=True, feature_range=(0, 1))



In [24]:
# Create dictionary called 'complex' with nodes, edges and meta-information
bleh = mapper.map(projected_X = projected_data, clusterer=cluster.DBSCAN(eps=0.3, min_samples=5), nr_cubes=30, overlap_perc=0.9)

# Visualize it
mapper.visualize(bleh, path_html="make_circles_keplermapper_output.html", 
                 title="make_circles(n_samples=5000, noise=0.03, factor=0.3)")

Mapping on data shaped (415, 2) using dimensions

Creating 27000 hypercubes.
There are 0 points in cube_0 / 27000 with starting range [ 0.  0.]
Cube_0 is empty.

There are 0 points in cube_1 / 27000 with starting range [ 0.          0.03333333]
Cube_1 is empty.

There are 0 points in cube_2 / 27000 with starting range [ 0.          0.06666667]
Cube_2 is empty.

There are 0 points in cube_3 / 27000 with starting range [ 0.   0.1]
Cube_3 is empty.

There are 0 points in cube_4 / 27000 with starting range [ 0.          0.13333333]
Cube_4 is empty.

There are 0 points in cube_5 / 27000 with starting range [ 0.          0.16666667]
Cube_5 is empty.

There are 0 points in cube_6 / 27000 with starting range [ 0.   0.2]
Cube_6 is empty.

There are 0 points in cube_7 / 27000 with starting range [ 0.          0.23333333]
Cube_7 is empty.

There are 0 points in cube_8 / 27000 with starting range [ 0.          0.26666667]
Cube_8 is empty.

There are 1 points in cube_9 / 27000 with starting range [

In [25]:
bleh.keys()

['nodes', 'meta', 'links']

In [43]:
nodes = bleh['nodes']
links = defaultdict(list)

In [44]:
# for k in nodes:
#     for kn in nodes:
#         if len(nodes[k] + nodes[kn]) != len(set(nodes[kn] + nodes[k])): # there are non-unique id's in the union
#             links[k].append( kn )
#         '''    
#         if self.link_local:
#             if k.split("_")[0] == kn.split("_")[0]:
#         '''  

In [45]:
candidates = itertools.combinations(nodes.keys(),2)
for candidate in candidates:
    if len(nodes[candidate[0]]+nodes[candidate[1]]) != len(set(nodes[candidate[0]]+nodes[candidate[1]])):
        links[candidate[0]].append( candidate[1] )

In [46]:
len(links.keys())

67

In [47]:
len(bleh['links'].keys())

67

In [48]:
nodes.keys()

['4_138_0.0_[ 4 18]_[ 0.13333333  0.6       ]',
 '19_589_0.0_[19 19]_[ 0.63333333  0.63333333]',
 '23_706_0.0_[23 16]_[ 0.76666667  0.53333333]',
 '18_557_0.0_[18 17]_[ 0.6         0.56666667]',
 '7_214_0.0_[7 4]_[ 0.23333333  0.13333333]',
 '4_139_0.0_[ 4 19]_[ 0.13333333  0.63333333]',
 '5_162_0.0_[ 5 12]_[ 0.16666667  0.4       ]',
 '5_158_0.0_[5 8]_[ 0.16666667  0.26666667]',
 '7_213_0.0_[7 3]_[ 0.23333333  0.1       ]',
 '6_196_0.0_[ 6 16]_[ 0.2         0.53333333]',
 '22_674_0.0_[22 14]_[ 0.73333333  0.46666667]',
 '23_703_0.0_[23 13]_[ 0.76666667  0.43333333]',
 '23_705_0.0_[23 15]_[ 0.76666667  0.5       ]',
 '18_560_0.0_[18 20]_[ 0.6         0.66666667]',
 '5_172_0.0_[ 5 22]_[ 0.16666667  0.73333333]',
 '5_169_0.0_[ 5 19]_[ 0.16666667  0.63333333]',
 '6_194_0.0_[ 6 14]_[ 0.2         0.46666667]',
 '4_132_0.0_[ 4 12]_[ 0.13333333  0.4       ]',
 '7_228_0.0_[ 7 18]_[ 0.23333333  0.6       ]',
 '23_701_0.0_[23 11]_[ 0.76666667  0.36666667]',
 '17_527_0.0_[17 17]_[ 0.56666667  0.5